In [1]:
# limpar versões problemáticas
!pip uninstall -y dlib dlib-bin

# toolchain
!apt-get update -y
!apt-get install -y build-essential cmake libgl1 libglib2.0-0

# pip atualizado
!pip install -U pip setuptools wheel

# instalar dlib CPU-only
!pip install dlib-bin==19.24.2.post1

# verificar
import dlib
print("dlib", dlib.__version__, "| CUDA:", dlib.DLIB_USE_CUDA)

Found existing installation: dlib-bin 19.24.2.post1
Uninstalling dlib-bin-19.24.2.post1:
  Successfully uninstalled dlib-bin-19.24.2.post1
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,097 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading 

In [2]:
from google.colab import files
uploaded = files.upload()

Saving APS_Biometria_Secao6_Pacote.zip to APS_Biometria_Secao6_Pacote (2).zip


In [3]:
!unzip -o APS_Biometria_Secao6_Pacote.zip -d /content/
%cd /content/APS_Biometria
!find . -maxdepth 2 -type d -print

Archive:  APS_Biometria_Secao6_Pacote.zip
  inflating: /content/APS_Biometria/requirements.txt  
  inflating: /content/APS_Biometria/config.py  
  inflating: /content/APS_Biometria/modules/capture.py  
  inflating: /content/APS_Biometria/modules/preprocess.py  
  inflating: /content/APS_Biometria/modules/detect.py  
  inflating: /content/APS_Biometria/modules/features.py  
  inflating: /content/APS_Biometria/modules/recognize.py  
  inflating: /content/APS_Biometria/main.py  
/content/APS_Biometria
.
./datasets
./datasets/embeddings
./datasets/cadastro
./modules
./modules/__pycache__
./__pycache__


In [4]:
# remove dlib do requirements para evitar conflito
!sed -i '/^ *dlib\(==[^ ]*\)\? *$/d' requirements.txt

!pip install -q -r requirements.txt

In [5]:
!wget -q http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!wget -q http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2

!bunzip2 -f shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 -f dlib_face_recognition_resnet_model_v1.dat.bz2

!ls -lh *.dat

-rw-r--r-- 1 root root 22M Feb 11  2017 dlib_face_recognition_resnet_model_v1.dat
-rw-r--r-- 1 root root 96M Jul 24  2015 shape_predictor_68_face_landmarks.dat


In [6]:
%%writefile /content/APS_Biometria/main_colab_utils.py

Overwriting /content/APS_Biometria/main_colab_utils.py


In [7]:
import sys
sys.path.append('/content/APS_Biometria')

In [8]:
%%writefile /content/APS_Biometria/modules/detect.py
import cv2
import dlib
import numpy as np

class FaceDetector:
    def __init__(self):
        self.detector = dlib.get_frontal_face_detector()

    def detect(self, frame_bgr):
        if frame_bgr is None:
            return []

        # garantir memória contínua
        frame_bgr = np.ascontiguousarray(frame_bgr, dtype=np.uint8)

        # converter para grayscale (mais estável no Colab)
        if len(frame_bgr.shape) == 3:
            gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
        else:
            gray = frame_bgr

        gray = np.ascontiguousarray(gray, dtype=np.uint8)

        rects = self.detector(gray, 1)
        return rects

Overwriting /content/APS_Biometria/modules/detect.py


In [9]:
import importlib
import modules.detect
importlib.reload(modules.detect)
print("Detector recarregado com sucesso")

Detector recarregado com sucesso


In [10]:
from google.colab import files
import cv2
from modules.detect import FaceDetector
from modules.preprocess import Preprocessor
from modules.features import FaceEmbedder
from modules.recognize import RecognizerDB

det = FaceDetector()
pre = Preprocessor()
emb = FaceEmbedder()
db = RecognizerDB()

print("Envie 5 a 12 fotos do mesmo usuário...")
uploaded = files.upload()

embeddings = []

for name in uploaded.keys():
    img = cv2.imread(name)

    if img is None:
        print(name, "→ erro ao ler imagem")
        continue

    # normalização obrigatória para dlib
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    rects = det.detect(img)

    if len(rects) != 1:
        print(name, "→ precisa ter exatamente 1 rosto")
        continue

    aligned = pre.align_face(img, rects[0])

    if aligned is None:
        print(name, "→ falha no alinhamento")
        continue

    embeddings.append(emb.get_embedding(aligned))
    print(name, "→ OK")

if embeddings:
    db.save_user("julio", embeddings)
    print("✅ Usuário cadastrado")
else:
    print("❌ Nenhuma imagem válida")

Envie 5 a 12 fotos do mesmo usuário...


Saving Julio-image1.png to Julio-image1.png
Saving Julio-image2.png to Julio-image2.png
Saving Julio-image3.png to Julio-image3.png
Saving Julio-image4.png to Julio-image4.png
Saving Julio-image5.png to Julio-image5.png
Julio-image1.png → OK
Julio-image2.png → OK
Julio-image3.png → OK
Julio-image4.png → OK
Julio-image5.png → OK
✅ Usuário cadastrado


In [11]:
from google.colab import files
import cv2
from modules.detect import FaceDetector
from modules.preprocess import Preprocessor
from modules.features import FaceEmbedder
from modules.recognize import RecognizerDB

det = FaceDetector()
pre = Preprocessor()
emb = FaceEmbedder()
db = RecognizerDB()

print("Envie uma foto para autenticação...")
uploaded = files.upload()

for name in uploaded.keys():
    img = cv2.imread(name)
    rects = det.detect(img)

    if len(rects) != 1:
        print(name, "→ falhou (precisa ter 1 rosto)")
        continue

    aligned = pre.align_face(img, rects[0])
    probe = emb.get_embedding(aligned)

    user, dist = db.recognize(probe)

    if user:
        print("✅ Reconhecido:", user, "| distância:", round(dist, 3))
    else:
        print("❌ Não reconhecido | distância:", round(dist, 3))

Envie uma foto para autenticação...


Saving Julio-image1.png to Julio-image1 (1).png
✅ Reconhecido: julio | distância: 0.0
